<a href="https://colab.research.google.com/github/gulnuravci/spongebob_character_identifier/blob/main/spongebob_character_identifier_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [2]:
try:
  import gradio as gr
except:
  !pip -q install gradio
  import gradio as gr

print(f"Gradio version: {gr.__version__}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 

In [3]:
import matplotlib.pyplot as plt
import torch
import torchvision
import pandas as pd
import random
import shutil
import os

from pathlib import Path
from torch import nn
from torchvision import transforms
from timeit import default_timer as timer
from PIL import Image
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import List, Dict, Tuple

try:
    from torchinfo import summary
except:
    !pip install -q torchinfo
    from torchinfo import summary

try:
  import torchmetrics
except:
  !pip install -q torchmetrics
  import torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 6.3 MB/s eta 0:00:00


In [4]:
# Import necessary spongebob_character_identifier files from github
try:
  import data_setup, engine, model_builder, transfer_learning_model_builder, utils
except:
  !git clone https://github.com/gulnuravci/spongebob_character_identifier
  !cp spongebob_character_identifier/spongebob_character_identifier/data_setup.py .
  !cp spongebob_character_identifier/spongebob_character_identifier/engine.py .
  !cp spongebob_character_identifier/spongebob_character_identifier/model_builder.py .
  !cp spongebob_character_identifier/spongebob_character_identifier/transfer_learning_model_builder.py .
  !cp spongebob_character_identifier/spongebob_character_identifier/utils.py .
  !rm -rf spongebob_character_identifier
  import data_setup, engine, model_builder, transfer_learning_model_builder, utils

Cloning into 'spongebob_character_identifier'...
remote: Enumerating objects: 1820, done.
remote: Counting objects: 100% (380/380), done.
remote: Compressing objects: 100% (348/348), done.
remote: Total 1820 (delta 33), reused 354 (delta 22), pack-reused 1440
Receiving objects: 100% (1820/1820), 220.51 MiB | 21.26 MiB/s, done.
Resolving deltas: 100% (118/118), done.
Updating files: 100% (1492/1492), done.


# set up device

In [5]:
device = utils.setup_target_device(device="cuda")

Current device: cuda



# get data and set up directories

In [6]:
# Import data
DATA_NAME = "character_images"
GITHUB_URL = "https://github.com/gulnuravci/spongebob_character_identifier/raw/main/character_images.zip"
data_setup.import_data_from_github(data_name=DATA_NAME, github_raw_url=GITHUB_URL)

# Setup directories
data_dir = Path("data") / DATA_NAME
train_dir = data_dir / "train"
test_dir = data_dir / "test"

data/character_images directory does not exist, importing...
Import complete.


In [7]:
import os

# Define the directory path
directory_path = "/content/models"

# Create the directory if it doesn't exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Check if the directory has been created
if os.path.exists(directory_path):
    print(f"Directory '{directory_path}' has been created.")
else:
    print(f"Failed to create directory '{directory_path}'.")


Directory '/content/models' has been created.


# efficientnet_b2

## create model and get transform


In [8]:
from torchvision.models import efficientnet_b2, EfficientNet_B2_Weights

# Set the name of the model to save
MODEL_NAME = "model_efficientnet_b2"
NOTES = "Using transfer learning with PyTorch pre-trained model 'efficientnet_b2'"

# Setup hyperparameters
NUM_EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.001
OUT_FEATURES = 10

# Create model
effnetb2 = transfer_learning_model_builder.create_effnetb2(out_features=OUT_FEATURES,
                                                           device=device)

# Get transform
weights = EfficientNet_B2_Weights.DEFAULT
auto_transforms = weights.transforms()

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-bcdf34b7.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-bcdf34b7.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 59.6MB/s]


[INFO] Created new effnetb2 model.


## create dataloaders, loss, and optimizer

In [9]:
# Create training and testing DataLoaders as well as get a list of class names
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               train_transform=auto_transforms,
                                                                               test_transform=auto_transforms,
                                                                               batch_size=BATCH_SIZE,
                                                                               device=device)

In [10]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(effnetb2.parameters(),
                             lr=LEARNING_RATE)

## train

In [11]:
# Start the timer
start_time = timer()

# Setup training and save the results
effnetb2_results = engine.train(model=effnetb2,
                             train_dataloader=train_dataloader,
                             test_dataloader=test_dataloader,
                             optimizer=optimizer,
                             loss_fn=loss_fn,
                             epochs=NUM_EPOCHS,
                             device=device,
                             writer=utils.create_writer(experiment_name="original",
                                                 model_name=MODEL_NAME,
                                                 extra=f"{NUM_EPOCHS}_epochs"))

# End the timer and print out how long it took
end_time = timer()
effnetb2_total_time = end_time-start_time

[INFO] Created SummaryWriter, saving to: runs/2023-11-14/original/model_efficientnet_b2/10_epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.8288 | train_acc: 0.5760 | test_loss: 1.2592 | test_acc: 0.9258
Epoch: 2 | train_loss: 1.0018 | train_acc: 0.9437 | test_loss: 0.7517 | test_acc: 0.9648
Epoch: 3 | train_loss: 0.6447 | train_acc: 0.9719 | test_loss: 0.5141 | test_acc: 0.9648
Epoch: 4 | train_loss: 0.4629 | train_acc: 0.9646 | test_loss: 0.3970 | test_acc: 0.9570
Epoch: 5 | train_loss: 0.3655 | train_acc: 0.9750 | test_loss: 0.3269 | test_acc: 0.9648
Epoch: 6 | train_loss: 0.2934 | train_acc: 0.9781 | test_loss: 0.2777 | test_acc: 0.9688
Epoch: 7 | train_loss: 0.2422 | train_acc: 0.9833 | test_loss: 0.2446 | test_acc: 0.9766
Epoch: 8 | train_loss: 0.2130 | train_acc: 0.9906 | test_loss: 0.2260 | test_acc: 0.9727
Epoch: 9 | train_loss: 0.1930 | train_acc: 0.9865 | test_loss: 0.2029 | test_acc: 0.9688
Epoch: 10 | train_loss: 0.1757 | train_acc: 0.9875 | test_loss: 0.1911 | test_acc: 0.9727


## save model

In [12]:
# Save the model
utils.save_model_with_hyperparameters(model=effnetb2,
                                      model_results=effnetb2_results,
                                      target_dir="models",
                                      model_name=MODEL_NAME,
                                      num_epochs=NUM_EPOCHS,
                                      batch_size=BATCH_SIZE,
                                      hidden_units="N/A",
                                      learning_rate=LEARNING_RATE,
                                      image_size="224x224",
                                      train_dataloader_length=len(train_dataloader),
                                      test_dataloader_length=len(test_dataloader),
                                      notes=NOTES)

# deployment

## to gradio (temporary)

In [13]:
# Put EffNetB2 on CPU
effnetb2.to("cpu")

# Check the device
next(iter(effnetb2.parameters())).device

device(type='cpu')

In [14]:
def predict(img) -> Tuple[Dict, float]:
  """Transforms and performs a prediction on img and returns prediction and time taken.
  """
  # Start the timer
  start_time = timer()

  # Transform the target image and add a batch dimension
  img = auto_transforms(img).unsqueeze(dim=0)

  # Put model into evaluation mode and turn on inference mode
  effnetb2.eval()
  with torch.inference_mode():
    # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
    pred_probs = torch.softmax(effnetb2(img), dim=1)

  # Create a prediction label and prediction probability dictionary for each prediction class (required format for Gradio's output parameter)
  pred_labels_and_probs = {class_names[i]:float(pred_probs[0][i]) for i in range(len(class_names))}

  # Calculate the prediction time
  pred_time = round(timer() - start_time)

  # Return the prediction dictionary and prediction time
  return pred_labels_and_probs, pred_time

In [15]:
# Get a list of all test image filepaths
test_data_paths = list(Path(test_dir).glob("*/*.jpg"))

# Randomly select a test image path
random_image_path = random.sample(test_data_paths, k=1)[0]

# Open the target image
image = Image.open(random_image_path)
print(f"[INFO] Predicting on image at path: {random_image_path}\n")

# Predict on the target image and print out the outputs
pred_dict, pred_time = predict(img=image)
print(f'Prediction label and probability dictionary: \n{pred_dict}')
print(f"Prediction time: {pred_time} seconds")

[INFO] Predicting on image at path: data/character_images/test/patrick_star/patrick_star_86.jpg

Prediction label and probability dictionary: 
{'eugene_h_krabs': 0.015761101618409157, 'gary_the_snail': 0.004570010583847761, 'karen_plankton': 0.0426739901304245, 'mrs_puff': 0.014805634506046772, 'patrick_star': 0.8181440234184265, 'pearl_krabs': 0.016805436462163925, 'sandy_cheeks': 0.030565641820430756, 'sheldon_j_plankton': 0.006081614177674055, 'spongebob_squarepants': 0.03927656635642052, 'squidward_tentacles': 0.011315987445414066}
Prediction time: 0 seconds


In [16]:
# Create a list of example inputs to our Gradio demo
example_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=3)]
example_list

[['data/character_images/test/karen_plankton/karen_plankton_20.jpg'],
 ['data/character_images/test/sandy_cheeks/sandy_cheeks_12.jpg'],
 ['data/character_images/test/pearl_krabs/pearl_krabs_8.jpg']]

In [17]:
title = "Spongebob Chracter Identifier 🧽👖🐙🦀🐿️🍍🍔🐳🖥️"
description = "An EfficientNetB2 feature extractor computer vision model to classify between 10 character from Spongebob Squarepants: Spongebob, Patrick, Squidward, Gary, Mr. Krabs, Mrs.Puff, Sandy, Plankton, Karen, and Pearl"
article = "Read more at: [Spongebob Character Identifier](https://gulnuravci.github.io/scripts/project_pages/spongebob_identifier.html)"
demo = gr.Interface(fn=predict, # mapping function from input to output
                    inputs=gr.Image(type="pil"), # what are the inputs?
                    outputs=[gr.Label(num_top_classes=10, label="Predictions"), # what are the outputs?
                             gr.Number(label="Prediction time (s)")], # our fn has two outputs, therefore we have two outputs
                    examples=example_list,
                    title=title,
                    description=description,
                    article=article)

demo.launch(debug=False, # print errors locally?
            share=True) # generate a publically shareable URL?

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4b05c141b12739cedb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## to hugging face spaces (permanent)

In [18]:
# Create demo path
spongebob_character_identifier_path = Path("demos/spongebob_character_identifier/")

# Remove files that might already exist and create new directory
if spongebob_character_identifier_path.exists():
  shutil.rmtree(spongebob_character_identifier_path)
  spongebob_character_identifier_path.mkdir(parents=True, # make the parent folders?
                                            exist_ok=True) # create it even if it already exists?
else:
  # If the file doesn't exist, create it anyway
  spongebob_character_identifier_path.mkdir(parents=True, # make the parent folders?
                                            exist_ok=True) # create it even if it already exists?

# Check what's in the folder
!ls demos/spongebob_character_identifier/

In [19]:
# Create an examples directory
spongebob_character_identifier_examples_path = spongebob_character_identifier_path / "examples"
spongebob_character_identifier_examples_path.mkdir(parents=True, exist_ok=True)

# Collect three random test dataset image paths
spongebob_character_identifier_examples = [Path('data/character_images/test/gary_the_snail/gary_the_snail_54.jpg'),
                                           Path('data/character_images/test/squidward_tentacles/squidward_tentacles_101.jpg'),
                                           Path('data/character_images/test/pearl_krabs/pearl_krabs_45.jpg')]

# Copy the three random images to the examples directory
for example in spongebob_character_identifier_examples:
  destination = spongebob_character_identifier_examples_path / example.name
  print(f"[INFO] Copying {example} to {destination}")
  shutil.copy2(src=example, dst=destination)

[INFO] Copying data/character_images/test/gary_the_snail/gary_the_snail_54.jpg to demos/spongebob_character_identifier/examples/gary_the_snail_54.jpg
[INFO] Copying data/character_images/test/squidward_tentacles/squidward_tentacles_101.jpg to demos/spongebob_character_identifier/examples/squidward_tentacles_101.jpg
[INFO] Copying data/character_images/test/pearl_krabs/pearl_krabs_45.jpg to demos/spongebob_character_identifier/examples/pearl_krabs_45.jpg


In [20]:
# get example filepaths in a list of lists
example_list = [["examples/" + example] for example in os.listdir(spongebob_character_identifier_examples_path)]
example_list

[['examples/gary_the_snail_54.jpg'],
 ['examples/pearl_krabs_45.jpg'],
 ['examples/squidward_tentacles_101.jpg']]

In [21]:
# Create a source path for our target model
effnetb2_spongebob_character_identifier_model_path = "models/model_efficientnet_b2.pth"

# Create a destination path for our target model
effnetb2_spongebob_character_identifier_model_destination = spongebob_character_identifier_path / effnetb2_spongebob_character_identifier_model_path.split("/")[1]

# Try to move the file
try:
  print(f"[INFO] Attempting to move {effnetb2_spongebob_character_identifier_model_path} to {effnetb2_spongebob_character_identifier_model_destination}")

  # Move the model
  shutil.move(src=effnetb2_spongebob_character_identifier_model_path,
              dst=effnetb2_spongebob_character_identifier_model_destination)

  print(f"[INFO] Model move complete.")

# If the model has already been moved, check if it exists
except:
  print(f"[INFO] No model found at {effnetb2_spongebob_character_identifier_model_path}, perhaps its already been move?")
  print(f"[INFO] Model exists at {effnetb2_spongebob_character_identifier_model_destination} : {effnetb2_spongebob_character_identifier_model_destination.exists()}")

[INFO] Attempting to move models/model_efficientnet_b2.pth to demos/spongebob_character_identifier/model_efficientnet_b2.pth
[INFO] Model move complete.


In [22]:
%%writefile demos/spongebob_character_identifier/model.py

import torch
import torchvision

from torch import nn
from torchvision.models import efficientnet_b2, EfficientNet_B2_Weights
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url

def create_effnetb2_model(num_classes:int=10,
                          seed:int=42):
  """Creates an EficientNetB2 feature extractor model and transforms.

  Args:
    num_classes (int, optional): number of classes in the classifier head. Defaults to 10.
    seed (int, optional): random seed value. Defaults to 42.

  Returns:
    model (torch.nn.Module): EffNetB2 feature extractor model.
    transforms (torchvision.transforms): EfnetB2 image transforms.
  """
  # Fix for wrong hash error from: https://github.com/pytorch/vision/issues/7744
  def get_state_dict(self, *args, **kwargs):
      kwargs.pop("check_hash")
      return load_state_dict_from_url(self.url, *args, **kwargs)
  WeightsEnum.get_state_dict = get_state_dict

  # Create EffNetB2 pretrained weights, transforms and model
  weights = EfficientNet_B2_Weights.DEFAULT
  transforms = weights.transforms()
  model = efficientnet_b2(weights=weights)

  # Freeze all layers in base model
  for param in model.features.parameters():
        param.requires_grad = False

  # Change the classifier head with random seed for reproducibility
  torch.manual_seed(seed)
  model.classifier = nn.Sequential(
      nn.Dropout(p=0.3),
      nn.Linear(in_features=1408, out_features=num_classes)
  )

  return model, transforms


Writing demos/spongebob_character_identifier/model.py


In [23]:
%%writefile demos/spongebob_character_identifier/app.py

### Imports and class names setup ###
import gradio as gr
import os
import torch

from model import create_effnetb2_model
from timeit import default_timer as timer
from typing import Tuple, Dict

# Setup class names
class_names = ["eugene_h_krabs", "gary_the_snail", "karen_plankton", "mrs_puff", "patrick_star", "pearl_krabs", "sandy_cheeks", "sheldon_j_plankton", "spongebob_squarepants", "squidward_tentacles"]

### Model and transforms preparation ###
# Create EffNetB2 model
effnetb2, effnetb2_transforms = create_effnetb2_model(
    num_classes=10
)

# Load saved weights
effnetb2.load_state_dict(
    torch.load(
        f="model_efficientnet_b2.pth",
        map_location=torch.device("cpu")
    )
)

### Predict function ###

# Create predict function
def predict(img) -> Tuple[Dict, float]:
  """Transforms and performs a prediction on img and returns prediction and time taken.
  """
  # Start the timer
  start_time = timer()

  # Transform the target image and add a batch dimension
  img = effnetb2_transforms(img).unsqueeze(dim=0)

  # Put model into evaluation mode and turn on inference mode
  effnetb2.eval()
  with torch.inference_mode():
    # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
    pred_probs = torch.softmax(effnetb2(img), dim=1)

  # Create a prediction label and prediction probability dictionary for each prediction class (required format for Gradio's output parameter)
  pred_labels_and_probs = {class_names[i]:float(pred_probs[0][i]) for i in range(len(class_names))}

  # Calculate the prediction time
  pred_time = round(timer() - start_time)

  # Return the prediction dictionary and prediction time
  return pred_labels_and_probs, pred_time

### Gradio app ###
title = "Spongebob Chracter Identifier 🧽👖🐙🦀🐿️🍍🍔🐳🖥️"
description = "An EfficientNetB2 feature extractor computer vision model to classify between 10 character from Spongebob Squarepants: Spongebob, Patrick, Squidward, Gary, Mr. Krabs, Mrs.Puff, Sandy, Plankton, Karen, and Pearl"
article = "Read more at: [Spongebob Character Identifier](https://gulnuravci.github.io/scripts/project_pages/spongebob_identifier.html)"

# Create examples list from "examples/" directory
example_list = [["examples/" + example] for example in os.listdir("examples")]

demo = gr.Interface(fn=predict, # mapping function from input to output
                    inputs=gr.Image(type="pil"), # what are the inputs?
                    outputs=[gr.Label(num_top_classes=10, label="Predictions"), # what are the outputs?
                             gr.Number(label="Prediction time (s)")], # our fn has two outputs, therefore we have two outputs
                    examples=example_list,
                    title=title,
                    description=description,
                    article=article)

demo.launch()

Writing demos/spongebob_character_identifier/app.py


In [24]:
%%writefile demos/spongebob_character_identifier/requirements.txt
torch>=1.12.0
torchvision>=0.13.0
gradio>=3.1.4

Writing demos/spongebob_character_identifier/requirements.txt


In [25]:
!ls demos/spongebob_character_identifier

app.py	examples  model_efficientnet_b2.pth  model.py  requirements.txt


In [26]:
# Change into and then zip the spongebob_character_identifier folder but exclude certain files
!cd demos/spongebob_character_identifier && zip -r ../spongebob_character_identifier.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

# Download the zipped app
try:
  from google.colab import files
  files.download("demos/spongebob_character_identifier.zip")
except:
  print("Not running in Google Colab, can't use google.colab.files.download(), please manually download.")

  adding: app.py (deflated 54%)
  adding: examples/ (stored 0%)
  adding: examples/gary_the_snail_54.jpg (deflated 1%)
  adding: examples/pearl_krabs_45.jpg (deflated 3%)
  adding: examples/squidward_tentacles_101.jpg (deflated 2%)
  adding: model_efficientnet_b2.pth (deflated 8%)
  adding: model.py (deflated 55%)
  adding: requirements.txt (deflated 4%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
from IPython.display import IFrame

IFrame(src="https://huggingface.co/spaces/gulnuravci/spongebob_character_identifier", width=900, height=750)